In [152]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import PunktSentenceTokenizer
from nltk.stem import SnowballStemmer, LancasterStemmer

In [122]:
df = pd.read_csv("coffee_fix.csv")
df.head()

,slug,all_text,rating,roaster,name,location,origin,roast,est_price,review_date,agtron,aroma,acid,body,flavor,aftertaste,with_milk,desc_1,desc_2,desc_3
0,https://www.coffeereview.com/review/sweety-esp...,\n\n\n95\n\n\nA.R.C.\n“Sweety” Espresso Blend...,95,A.R.C.,“Sweety” Espresso Blend,"Hong Kong, China",Panama; Ethiopia,Medium-Light,HKD $250/227 grams,November 2017,50/73,9.0,NaN,9.0,9.0,9.0,9.0,"Evaluated as espresso. Sweet-toned, deeply ric...",An espresso blend comprised of coffees from Pa...,A radiant espresso blend that shines equally i...
1,https://www.coffeereview.com/review/flora-blen...,\n\n\n94\n\n\nA.R.C.\nFlora Blend Espresso\n\...,94,A.R.C.,Flora Blend Espresso,"Hong Kong, China",Africa; Asia Pacific,Medium-Light,HKD $158/227 grams,November 2017,54/77,9.0,NaN,9.0,9.0,8.0,9.0,"Evaluated as espresso. Sweetly tart, floral-to...",An espresso blend comprised of coffees from Af...,"A floral-driven straight shot, amplified with ..."
2,https://www.coffeereview.com/review/ethiopia-s...,\n\n\n92\n\n\nRevel Coffee\nEthiopia Shakiso ...,92,Revel Coffee,Ethiopia Shakiso Mormora,"Billings, Montana","Guji Zone, southern Ethiopia",Medium-Light,$16.00/12 ounces,November 2017,54/70,9.0,8.0,8.0,9.0,8.0,NaN,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce..."
3,https://www.coffeereview.com/review/ethiopia-s...,\n\n\n92\n\n\nRoast House\nEthiopia Suke Quto...,92,Roast House,Ethiopia Suke Quto,"Spokane, Washington","Guji Zone, Oromia Region, south-central Ethiopia",Medium-Light,$19.00/16 ounces,November 2017,53/79,8.0,8.0,9.0,9.0,8.0,NaN,"Delicate, sweetly spice-toned. Pink peppercorn...",This coffee tied for the third-highest rating ...,Lavender-like flowers and hints of zesty pink ...
4,https://www.coffeereview.com/review/ethiopia-g...,\n\n\n94\n\n\nBig Creek Coffee Roasters\nEthi...,94,Big Creek Coffee Roasters,Ethiopia Gedeb Halo Beriti,"Hamilton, Montana","Gedeb District, Gedeo Zone, southern Ethiopia",Medium,$16.50/12 ounces,November 2017,48/70,9.0,9.0,9.0,9.0,8.0,NaN,"Deeply sweet, subtly pungent. Honey, pear, tan...",Southern Ethiopia coffees like this one are pr...,A deeply and generously lush cup saved from se...


In [124]:
tokens = [word_tokenize(i) for i in df["desc_1"]]
t1 = tokens[0]

In [125]:
t1

['Evaluated',
 'as',
 'espresso',
 '.',
 'Sweet-toned',
 ',',
 'deeply',
 'rich',
 ',',
 'chocolaty',
 '.',
 'Vanilla',
 'paste',
 ',',
 'dark',
 'chocolate',
 ',',
 'narcissus',
 ',',
 'pink',
 'grapefruit',
 'zest',
 ',',
 'black',
 'cherry',
 'in',
 'aroma',
 'and',
 'cup',
 '.',
 'Plush',
 ',',
 'syrupy',
 'mouthfeel',
 ';',
 'resonant',
 ',',
 'flavor-saturated',
 'finish',
 '.',
 'In',
 'three',
 'parts',
 'milk',
 ',',
 'rich',
 'chocolate',
 'tones',
 'intensify',
 ',',
 'along',
 'with',
 'intimations',
 'of',
 'vanilla',
 'paste',
 'and',
 'black',
 'cherry',
 'in',
 'the',
 'short',
 'finish',
 'and',
 'floral-toned',
 'citrus',
 'zest',
 'in',
 'the',
 'long',
 '.']

In [126]:
count = 0
for i in df["desc_1"]:
    if "three parts milk" in i:
        #print(i)
        count += 1
print(count)

210


In [127]:
def process_content(description):
    l = []
    try:
        words = nltk.word_tokenize(description.lower().replace("-"," "))
        #print(words)
        tagged = nltk.pos_tag(words)
        #print(tagged)

        # All chunks
        chunk_gram = r"""Chunk: {<NNP>?<RB.?>*<VBD>*<JJ.?>*<NNP.?>*<NN.?>*<NN.?>*}"""
        chunk_parser = nltk.RegexpParser(chunk_gram)
        chunked = chunk_parser.parse(tagged)

        # Finding the finish description
        finish_gram = r"""Chunk: {<NNP>?<RB.?>*<VBD>*<VBN>*<JJ.?>*<NNP.?>*<NN.?>*<NN.?>*<IN>+<DT>+<JJ.?>*<NN>*}"""
        finish_parser = nltk.RegexpParser(finish_gram)
        finish_chunks = finish_parser.parse(tagged)

        # Finding the milk description
        
        print(description)

        for chunk in chunked:
            if hasattr(chunk,"label"):
                if chunk.label() == "Chunk":
                    l.append(chunk)
            # print(chunk)
        print("---Finish---")
        if len(finish_chunks) == 0:
            for chunk in chunked:
                if hasattr(chunk,"label"):
                    if chunk.label() == "Chunk":
                        for j in chunk:
                            print(j[0])
        for chunk in finish_chunks:
            if hasattr(chunk,"label"):
                if chunk.label() == "Chunk":
                    print(chunk)
                    # for j in chunk:
                    #     print(j[0])
        print("---Finish done---")

    except Exception as e:
        print(str(e))

    return l

chunks = process_content(df["desc_1"][0])
chunks

Evaluated as espresso. Sweet-toned, deeply rich, chocolaty. Vanilla paste, dark chocolate, narcissus, pink grapefruit zest, black cherry in aroma and cup. Plush, syrupy mouthfeel; resonant, flavor-saturated finish. In three parts milk, rich chocolate tones intensify, along with intimations of vanilla paste and black cherry in the short finish and floral-toned citrus zest in the long. 
---Finish---
(Chunk black/JJ cherry/NN in/IN the/DT short/JJ finish/NN)
(Chunk toned/VBN citrus/NN zest/NN in/IN the/DT long/JJ)
---Finish done---


[Tree('Chunk', [('espresso', 'NN')]),
 Tree('Chunk', [('sweet', 'NN')]),
 Tree('Chunk', [('toned', 'VBD')]),
 Tree('Chunk', [('deeply', 'RB'), ('rich', 'JJ')]),
 Tree('Chunk', [('chocolaty', 'NN')]),
 Tree('Chunk', [('vanilla', 'NN'), ('paste', 'NN')]),
 Tree('Chunk', [('dark', 'JJ'), ('chocolate', 'NN')]),
 Tree('Chunk', [('narcissus', 'NN')]),
 Tree('Chunk', [('grapefruit', 'NN'), ('zest', 'NN')]),
 Tree('Chunk', [('black', 'JJ'), ('cherry', 'NN')]),
 Tree('Chunk', [('aroma', 'NN')]),
 Tree('Chunk', [('cup', 'NN')]),
 Tree('Chunk', [('plush', 'NN')]),
 Tree('Chunk', [('syrupy', 'JJ'), ('mouthfeel', 'NN')]),
 Tree('Chunk', [('resonant', 'NN')]),
 Tree('Chunk', [('flavor', 'NN')]),
 Tree('Chunk', [('saturated', 'VBD'), ('finish', 'JJ')]),
 Tree('Chunk', [('parts', 'NNS'), ('milk', 'NN')]),
 Tree('Chunk', [('rich', 'JJ'), ('chocolate', 'NN'), ('tones', 'NNS')]),
 Tree('Chunk', [('intimations', 'NNS')]),
 Tree('Chunk', [('vanilla', 'NN'), ('paste', 'NN')]),
 Tree('Chunk', [('black', 'JJ'

In [144]:
def clean_chunk(chunk):
    grammar = [i[1] for i in chunk]
    words = [i[0] for i in chunk]
    for i in ["finish","tone","tones","toned","mouthfeel","paste"]:
        for j in chunk:
            if j[0] == i:
                chunk.remove(j)
    for i in ["milk","short","long","cup","flavor","flavour"]:
        if i in words:
            return None
    return chunk

In [145]:
clean = [clean_chunk(i) for i in chunks if clean_chunk(i) is not None]
clean

[Tree('Chunk', [('espresso', 'NN')]),
 Tree('Chunk', [('sweet', 'NN')]),
 Tree('Chunk', []),
 Tree('Chunk', [('deeply', 'RB'), ('rich', 'JJ')]),
 Tree('Chunk', [('chocolaty', 'NN')]),
 Tree('Chunk', [('vanilla', 'NN')]),
 Tree('Chunk', [('dark', 'JJ'), ('chocolate', 'NN')]),
 Tree('Chunk', [('narcissus', 'NN')]),
 Tree('Chunk', [('grapefruit', 'NN'), ('zest', 'NN')]),
 Tree('Chunk', [('black', 'JJ'), ('cherry', 'NN')]),
 Tree('Chunk', [('aroma', 'NN')]),
 Tree('Chunk', [('plush', 'NN')]),
 Tree('Chunk', [('syrupy', 'JJ')]),
 Tree('Chunk', [('resonant', 'NN')]),
 Tree('Chunk', [('saturated', 'VBD')]),
 Tree('Chunk', [('rich', 'JJ'), ('chocolate', 'NN')]),
 Tree('Chunk', [('intimations', 'NNS')]),
 Tree('Chunk', [('vanilla', 'NN')]),
 Tree('Chunk', [('black', 'JJ'), ('cherry', 'NN')]),
 Tree('Chunk', [('floral', 'JJ')]),
 Tree('Chunk', [('citrus', 'NN'), ('zest', 'NN')])]

In [159]:
stemmer = PorterStemmer()
stemmer = SnowballStemmer(language = "english")

In [160]:
def extract_characteristics(raw):
    characteristics = []
    for chunk in raw:
        grammar = [i[1] for i in chunk]
        words = [i[0] for i in chunk]
        clean = clean_chunk(chunk)
        if clean is None:
            continue

        if len(clean) == 0:
            continue
        
        if len(clean) == 1:
            characteristics.append(stemmer.stem(words[0]))
            continue

        if all(i=="NN" for i in grammar):
            characteristics.append(stemmer.stem(words[0]))
            continue
        
        elif grammar[0].startswith('JJ') and grammar[1].startswith('NN'):
            characteristics.append(stemmer.stem(words[1]))
            continue

        elif grammar[0].startswith('RB') and grammar[1].startswith('JJ'):
            characteristics.append(stemmer.stem(words[1]))
            continue
    return characteristics

extract_characteristics(chunks)

['espresso',
 'sweet',
 'rich',
 'chocolati',
 'vanilla',
 'chocol',
 'narcissus',
 'grapefruit',
 'cherri',
 'aroma',
 'plush',
 'syrupi',
 'reson',
 'satur',
 'chocol',
 'intim',
 'vanilla',
 'cherri',
 'floral',
 'citrus']

In [ ]:
df